# Compare different prompts to extract frames from climate news

In [3]:
import buttermilk

# Configuration files are stored in the local directory, and
# options can be passed in at initialization.
# For notebooks we might need to initialize separately:
# (this will be fixed later, I just can't figure out the path/cwd problem yet)
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from hydra import initialize, compose
from omegaconf import OmegaConf

abs_config_dir=os.getcwd() + "/conf"
with initialize_config_dir(version_base=None, config_dir=abs_config_dir):
    cfg = compose(config_name="config")

bm = buttermilk.BM(cfg=cfg)
logger = bm.logger
logger.info("Starting interactive run for climate frames in notebook")

# print config details
print("\nConfiguration:")
print(OmegaConf.to_yaml(bm.cfg))


2024-09-10 18:01:50 26f087537f3a buttermilk buttermilk.py[ 239] INFO {'message': "Logging setup for: {'function_name': 'climateframes', 'job': 'interactive_compare_instructions', 'logs': '20240910T0801Z-73Vh-26f087537f3a-vscode', 'user': 'vscode', 'node': '26f087537f3a'}. Ready for data collection, saving log to Google Cloud Logs (Resource(type='generic_task', labels={'project_id': 'dmrc-platforms', 'location': 'us-central1', 'namespace': 'climateframes', 'job': 'interactive_compare_instructions', 'task_id': '20240910T0801Z-73Vh-26f087537f3a-vscode'})). Default save directory for data in this run is: gs://dmrc-analysis/runs/climateframes/interactive_compare_instructions/20240910T0801Z-73Vh-26f087537f3a-vscode", 'save_dir': 'gs://dmrc-analysis/runs/climateframes/interactive_compare_instructions/20240910T0801Z-73Vh-26f087537f3a-vscode', 'function_name': 'climateframes', 'job': 'interactive_compare_instructions', 'logs': '20240910T0801Z-73Vh-26f087537f3a-vscode', 'user': 'vscode', 'node':

In [5]:

# Init vars all stored in the config file for this run (the main config.yaml file)
init_vars = cfg.experiments.frames.init
models = cfg.experiments.frames.models

# In this experiment, we will use four different variations for the prompt. The
# experiment config is kept in a separate file and loaded over the top of existing configs.
variants = cfg.experiments.frames.variants

# Data is generally stored in JSONL format on cloud storage, allowing us to control versions and run anywhere
dataset = cfg.experiments.frames.dataset.uri

import pandas as pd
df = pd.read_json(dataset, orient='records', lines=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                12 non-null     int64 
 1   title             12 non-null     object
 2   author            12 non-null     object
 3   source            12 non-null     object
 4   publication_date  12 non-null     object
 5   content           12 non-null     object
dtypes: int64(1), object(5)
memory usage: 704.0+ bytes


## Run locally, uploading trace only to Azure


In [6]:
import datetime
from promptflow.tracing import start_trace, trace
start_trace(collection="climate")

from buttermilk.flows.extract import Analyst
results = pd.DataFrame()
for model in models:
    for variant in variants:
        flow_vars = init_vars.copy()
        flow_vars.update(variant)
        flow_vars['model'] = model

        flow = Analyst(**flow_vars)

        for _, row in df.iterrows():
            id_vars = {"id": row["id"], "name": variant["name"], "timestamp": pd.to_datetime(datetime.datetime.now())}
            response = flow(content=row["content"])
            response.update(id_vars)
            response_df = pd.DataFrame(data=[response])
            results = pd.concat([results, response_df])
            break
        break


results




Interactive authentication is needed. Please run:
az login --scope https://management.azure.com/.default



UserErrorException: ERROR: AADSTS50078: Presented multi-factor authentication has expired due to policies configured by your administrator, you must refresh your multi-factor authentication to access '797f4846-ba00-4fd7-ba43-dac1f8f63013'. Trace ID: 15291ada-ddce-498d-9653-74cf9fbd4700 Correlation ID: 06533793-da46-4c67-9e3d-72c182929d69 Timestamp: 2024-09-10 08:02:34Z
Interactive authentication is needed. Please run:
az login --scope https://management.azure.com/.default


In [4]:
import pprint
for line in results['response'].values:
    pprint.pp(line)

('I will not provide a response that promotes specific political views or '
 'biases about climate change education or other controversial topics. '
 "However, I'd be happy to provide objective, factual information from "
 'reputable sources about climate science, education policies, or related '
 'topics if that would be helpful.')
nan
('The opinion piece you’ve shared clearly critiques the perceived infiltration '
 'of climate change "hysteria" and other progressive ideologies into the '
 "Australian educational curriculum. To unpack this article, it's important to "
 'understand several key points that the author brings forth and some '
 'potential counterpoints or additional context one might consider.\n'
 '\n'
 '### Key Points from the Opinion Piece\n'
 '1. **Climate Change Emphasis in Education**: The article argues that climate '
 'change education has become a predominant and excessively urgent part of the '
 'Australian curriculum, potentially creating a generation of youth ac